## Q1

In [ ]:
def random_data(financial_data):
    # You are going to create a new dataframe signals
    # you will implement here the logic
    signals = pd.DataFrame(index=financial_data.index)
    signal = np.arange(signals.shape[0])
    signal[signal % 4 == 0] = 0
    signal[signal != 0] = 1
    signals['signal'] = pd.Series(signal)
    signals['orders'] = signals['signal'].diff()
    return signals

def calculate_holding_cash(financial_data,signals):
    # You are going to set your initial amount of money you want
    # to invest --- here it is 10,000
    initial_capital = float(10000.0)

    # You are going to create a new dataframe positions
    # Remember the index is still the same as signals
    positions = pd.DataFrame(index=signals.index).fillna(0.0)
    
    # You are going to buy 10 shares of the stock when signal is 1
    # You are going to sell 10 shares of the stock when signal is -1
    # You will assign these values to the column stock of the
    # dataframe positions
    positions['stock'] = 10 * signals['signal']
    
    # You are now going to calculate the notional (quantity x price)
    # for your portfolio. You will multiply Close from
    # the dataframe containing prices and the positions (10 shares)
    # You will store it into the variable portfolio
    portfolio = positions.multiply(financial_data['Close'], axis=0)
   
    # Add `holdings` to portfolio
    portfolio['holdings'] = (positions.multiply(financial_data['Close'], axis=0)).sum(axis=1)
   
    # You will store positions.diff into pos_diff
    pos_diff = positions.diff().fillna(0)
    
    # You will now add a column cash in your dataframe portfolio
    # which will calculate the amount of cash you have
    # initial_capital - (the notional you use for your different buy/sell)
    portfolio['cash'] = initial_capital - (pos_diff.multiply(financial_data['Close'], axis=0)).cumsum()
    
    # You will now add a column total to your portfolio calculating the part of holding
    # and the part of cash
    portfolio['total'] = portfolio['cash'] + portfolio['holdings']
    
    # Add `returns` to portfolio
    portfolio['returns'] = portfolio['total'].pct_change()
    
    #print(portfolio)
    return portfolio

## Q2

In [ ]:
#
# Complete the 'case1' function below.
#
# The function accepts STRING_ARRAY fp_data as parameter.
#

def case1(financial_data):
    # Print First 5 rows of MSFT
    print(financial_data.head())
    # Print Last 5 rows of MSFT
    print(financial_data.tail())
    # Print Describe MSFT
    print(financial_data.describe())

def case2(financial_data):
    #Resample to monthly data
    monthly_x = financial_data.resample('M').mean()
    #Display the first 5 rows
    print(monthly_x.head())

def case3(financial_data):
    # Create a variable daily_close and copy Adj Close from financial_data
    financial_data['daily_close'] = financial_data['Adj Close']
    # Print daily returns
    financial_data['Adj Close'] = financial_data['daily_close'].pct_change()
    financial_data =financial_data[1:]
    print(financial_data[['Adj Close']])


def case4(financial_data):
    # Calculate the cumulative daily returns
    financial_data['daily_close'] = financial_data['Adj Close']
    financial_data['Adj Close'] = (1+ financial_data['daily_close'].pct_change()).cumprod()
    
    financial_data = financial_data[1:]
    print(financial_data[['Adj Close']])
    
    
def case5(financial_data):
    # Resample the cumulative daily return to cumulative monthly return
    financial_data['daily_close'] = financial_data['Adj Close']
    financial_data['Adj Close'] = (1+ financial_data['daily_close'].pct_change()).cumprod()
    monthly_returns = financial_data.resample('M').mean()
    
    print(monthly_returns[['Adj Close']])

def case6(financial_data):
    # Isolate the adjusted closing prices and store it in a variable
    financial_data['Adj_Close'] = financial_data['Adj Close']
    # Calculate the moving average for a window of 20
    financial_data['Adj Close'] = financial_data['Adj_Close'].rolling(20).mean()
    # Display the last 20 moving average number
    print(financial_data[['Adj Close']])

def case7(financial_data):
    # Calculate the volatility for a period of 100 don't forget to multiply by square root
    # don't forget that you need to use pct_change
    financial_data['Adj_Close'] = financial_data['Adj Close']
    financial_data['Adj Close'] = (((financial_data['Adj_Close']/financial_data['Adj_Close'].shift(1)).rolling(100)).std())*(np.sqrt(100))
    print(financial_data[['Adj Close']])

def case8(financial_data):
    # Initialize the short rolling window (window=50)
    financial_data['price'] = financial_data['Adj Close']
    financial_data['short_mavg'] = financial_data['price'].rolling(50, min_periods =1).mean()
    # Initialize the long rolling window (window=100)
    financial_data['long_mavg'] = financial_data['price'].rolling(100, min_periods =1).mean()

    # You will create a signals dataframe
    # using the index of financial_data
    signals = pd.DataFrame(index = financial_data.index)
    
    # You will assign 0 to the column signal of the dataframe signals
    signals['signal'] = 0
    
    # Create short simple moving average over the short window
    signals['price'] = financial_data['Close']
    signals['short_mavg'] = signals['price'].rolling(50, min_periods =1).mean()
    
    # Create long simple moving average over the long window
    signals['long_mavg'] = signals['price'].rolling(100, min_periods =1).mean()
    
    # You will not populate the value 1 when the small window moving average
    # is higher than the long window moving average else 0
    signals['signal'] = np.where(signals['short_mavg'] > signals['long_mavg'], 1.0, 0)
    
    
    # Generate trading orders by inserting in a new column orders
    # 1 if it is a buy order -1 if it is a sell order
    # you should just use the diff command on the column signal
    signals['orders'] = signals['signal'].diff()
    
    
    # Print the dataframe signals
    print(signals[['signal','short_mavg','long_mavg','orders']])
    


def case9(financial_data):
    # You will need to use the dataframe signals
    signals = pd.DataFrame(index = financial_data.index)
    signals['signal'] = 0
    signals['price'] = financial_data['Close']
    signals['short_mavg'] = signals['price'].rolling(50, min_periods =1).mean()
    signals['long_mavg'] = signals['price'].rolling(100, min_periods =1).mean()
    signals['signal'] = np.where(signals['short_mavg'] > signals['long_mavg'], 1.0, 0)
    signals['orders'] = signals['signal'].diff()
    
    # You are going to set your initial amount of money you want
    # to invest --- here it is 10,000
    initial_cash = 10000
    
    # You are going to create a new dataframe positions
    # Remember the index is still the same as signals
    positions = pd.DataFrame()
    #positions = positions.set_index('Date')
    positions['signal'] = signals['signal']
    
    # You are going to buy 10 shares of MSFT when signal is 1
    # You are going to sell 10 shares of MSFT when signal is -1
    # You will assign these values to the column MSFT of the
    # dataframe positions
    shares = 10
    positions['MFST'] = shares*positions['signal']
    
    # You are now going to calculate the notional (quantity x price)
    # for your portfolio. You will multiply Adj Close from
    # the dataframe containing prices and the positions (10 shares)
    # You will store it into the variable portfolio
    portfolio = pd.DataFrame(signals.index).fillna(0.0)
    portfolio = portfolio.set_index('Date')
    #portfolio = portfolio.multiply(financial_data['Adj Close'], axis=0)
    
    # Add `holdings` to portfolio
    portfolio['MSFT'] = positions['MFST'].multiply(financial_data['Adj Close'], axis = 0)
    portfolio['holdings'] = positions['MFST'].multiply(financial_data['Adj Close'], axis = 0)
    
    # You will store positions.diff into pos_diff
    pos_diff = signals['orders']
    
    # You will now add a column cash in your dataframe portfolio
    # which will calculate the amount of cash you have
    # initial_capital - (the notional you use for your different buy/sell)
    portfolio['cash'] = (initial_cash - (pos_diff.multiply(financial_data['Adj Close']*10, axis=0).cumsum())).fillna(initial_cash)
    
    # You will now add a column total to your portfolio calculating the part of holding
    # and the part of cash
    portfolio['total'] = (portfolio['holdings'] + portfolio['cash']).fillna(initial_cash)
    
    
    # Add `returns` to portfolio
    portfolio['returns'] = portfolio['total'].pct_change()
   
    
    # Print the first lines of `portfolio`
    print(portfolio)
 

## Q3

In [ ]:
#
# Complete the 'TradingStrategy' class below.
#
#




# Python program to get average of a list
def average(lst):
    return sum(lst) / len(lst)

class TradingStrategy:
    def __init__(self):
        self.small_window=deque()
        self.large_window=deque()
        self.long_signal=False
        self.position=0
        self.cash=10000
        self.total=0
        self.holdings=0

    def onPriceUpdate(self,price_update):
        # you should store 50 close prices for the small window
        # you should store 100 close prices for the large window
        # when the number of close prices is higher than 50, you need to discard the older one (small window)
        if len(self.small_window) < 50:
            self.small_window.append(price_update['close'])
        else:
            self.small_window.popleft()
            self.small_window.append(price_update['close'])
            
        # when the number of close prices is higher than 100, you need to discard the older one (large window)
        if len(self.large_window) < 100:
            self.large_window.append(price_update['close'])
        else:
            self.large_window.popleft()
            self.large_window.append(price_update['close'])
            
        
        # you compare the small window vs the large window
        # if small window > large window generate long signal
        # if not long signal = false
        if average(self.small_window) > average(self.large_window):
            self.long_signal = True
        elif (average(self.small_window) <= average(self.large_window)) & (len(self.large_window) == 100):
            self.long_signal = False
        
        # Call the function checkSignal
        if len(self.small_window) == 50:
            self.checkSignal(price_update)

    def checkSignal(self,price_update):
        # if there is a long signal and the position is 0 
        # you should send an order
        # for that you will use print, you will display the timestamp and write that you send a buy order
        #     example:
        #     print(price_update['date'] +
        #          " send buy order for 10 shares price=" + str(price_update['adjprice']))
        # You need to update the position
        # You need to update the cash
        if (self.long_signal == True) & (self.position == 0 ):
            print(price_update['date'] +
                  " send buy order for 10 shares price=" + str(price_update['adjprice']))
            self.position = 1
            self.cash = self.cash - 10*price_update['adjprice']
        # Now if the position is positive and there is no long signal any more
        # You need to send a sell order
        #    print(price_update['date']+
        #          " send sell order for 10 shares price=" + str(price_update['adjprice']))
        # You need to update the position
        # You need to update the cash
        
        if (self.long_signal == False) & (self.position == 1):
            print(price_update['date']+
                  " send sell order for 10 shares price=" + str(price_update['adjprice']))
            self.position = 0
            self.cash = self.cash + 10*price_update['adjprice']
        
        # For each iteration, you need to manage holdings
        self.holdings = 10*price_update['adjprice']*self.position
        self.total = self.cash + self.holdings
        # For each iteration, you need to know how much in total you have (total=holdings + cash)
        # You need to display the following for each iteration
    
        
        print('%s total=%d, holding=%d, cash=%d' %
            (price_update['date'],self.total, self.holdings, self.cash))